In [31]:
import requests
import pandas as pd
import datetime

headers = {"User-Agent": "russ@sunriseanalysis.com"}

ticker = "AMD"
concept = "NetIncomeLoss"

tickers_json = requests.get(
    "https://www.sec.gov/files/company_tickers.json", headers=headers
)

cik_df = pd.DataFrame.from_dict(tickers_json.json(), orient="index")
cik_df["cik_str"] = cik_df["cik_str"].astype(str).str.zfill(10)
cik = cik_df[cik_df["ticker"] == ticker]["cik_str"].values[0]

concept_url = (
    f"https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/{concept}.json"
)

apple_ni = pd.DataFrame.from_dict(requests.get(concept_url, headers=headers).json())

apple_units = pd.DataFrame.from_dict(apple_ni["units"]["USD"])
for col in ["start", "end", "filed"]:
    apple_units[col] = pd.to_datetime(apple_units[col])
ap = apple_units
ap["time_delta"] = ap["end"] - ap["start"]

ap["val"] = ap["val"] / 1_000_000
ap.rename(columns={"val": "value(millions)"}, inplace=True)

annual_data = ap[ap["time_delta"] > pd.Timedelta(days=350)]
ap = ap.drop(annual_data.index)
annual_data = annual_data.drop_duplicates(subset="value(millions)", keep="last")
annual_data = annual_data.dropna(subset="frame")
annual_data = annual_data.drop(
    columns=["time_delta", "end", "start", "filed", "form", "fy", "fp"]
).reset_index(drop=True)

annual_data.rename(columns={"frame": "year"}, inplace=True)
extra_data = ap[ap["time_delta"] > pd.Timedelta(days=115)]
ap = ap.drop(extra_data.index)
extra_data.reset_index(drop=True, inplace=True)
ap = ap.drop_duplicates(subset="value(millions)", keep="last")
ap = ap.drop(columns=["time_delta", "filed", "form", "fy", "fp", "accn"])
ap = ap.dropna(subset=["frame"])
ap = ap.reset_index(drop=True)


def map_to_quarter(date):
    if 3 <= date.month <= 5:
        return 1
    elif 6 <= date.month <= 8:
        return 2
    elif 9 <= date.month <= 11:
        return 3
    else:  # 12, 1, 2
        return 4


ap["quarter"] = ap["end"].apply(map_to_quarter)

min_year = ap["end"].min().year
max_year = ap["end"].max().year

all_periods = pd.DataFrame(
    [
        (year, quarter)
        for year in range(min_year, max_year + 1)
        for quarter in range(1, 5)
    ],
    columns=["year", "quarter"],
)
ap["year"] = ap["end"].apply(lambda x: x.year)
merged = pd.merge(all_periods, ap, how="left", on=["year", "quarter"])
# Sort the data by 'Year' and 'quarter' to make sure it's in the correct order
merged.sort_values(["year", "quarter"], inplace=True)

# Create new columns 'next_start' and 'prev_end' to store the start and end dates of the next and previous quarters
merged["next_start"] = merged["start"].shift(-1)
merged["prev_end"] = merged["end"].shift()

# For missing quarters, set the 'start' date as the day after the end of the previous quarter,
# and the 'end' date as the day before the start of the next quarter
merged.loc[merged["start"].isna(), "start"] = merged["prev_end"] + pd.Timedelta(days=1)
merged.loc[merged["end"].isna(), "end"] = merged["next_start"] - pd.Timedelta(days=1)

# Drop the 'next_start' and 'prev_end' columns as they are no longer needed
merged.drop(["next_start", "prev_end"], axis=1, inplace=True)

today = datetime.datetime.today()

# Calculate the start date of the current quarter
if today.month < 4:
    quarter_start = datetime.datetime(today.year - 1, 12, 1)
elif today.month < 7:
    quarter_start = datetime.datetime(today.year, 3, 1)
elif today.month < 10:
    quarter_start = datetime.datetime(today.year, 6, 1)
else:
    quarter_start = datetime.datetime(today.year, 9, 1)

# Remove rows where the 'start' date is on or after the start of the current quarter
merged = merged[merged["start"] < quarter_start]

In [32]:
merged

,year,quarter,start,end,value(millions),frame
0,2011,1,2010-12-26,2011-04-02,510.0,CY2011Q1
1,2011,2,2011-04-03,2011-07-02,NaN,NaN
2,2011,3,2011-07-03,2011-10-01,97.0,CY2011Q3
3,2011,4,2011-10-02,2011-12-31,-177.0,CY2011Q4
4,2012,1,2012-01-01,2012-03-31,-590.0,CY2012Q1
5,2012,2,2012-04-01,2012-06-30,37.0,CY2012Q2
6,2012,3,2012-07-01,2012-09-29,-157.0,CY2012Q3
7,2012,4,2012-09-30,2012-12-29,-473.0,CY2012Q4
8,2013,1,2012-12-30,2013-03-30,-146.0,CY2013Q1
9,2013,2,2013-03-31,2013-06-29,-74.0,CY2013Q2
